In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler  # 导入标准化模块

In [27]:
# 定义神经网络模型
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)  # 第一个隐藏层
        self.fc2 = nn.Linear(16, 8)           # 第二个隐藏层
        self.fc3 = nn.Linear(8, output_size)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [28]:
data = pd.read_excel("使用版2012-2023-2个相关因素_2位小数.xlsx")

x, y = data.iloc[:,2:].to_numpy(), data.iloc[:,1].to_numpy().reshape(-1, 1)
scaler = MinMaxScaler()  # 实例化
X = scaler.fit_transform(x)  # 标准化特征
Y = scaler.fit_transform(y)  # 标准化标签

train_X, train_Y, val_X, val_Y = X[:-2], Y[:-2], X[-2:], Y[-2:]
train_torch_data = TensorDataset(torch.FloatTensor(train_X), torch.FloatTensor(train_Y))
val_torch_data = TensorDataset(torch.FloatTensor(val_X), torch.FloatTensor(val_Y))
all_torch_data = TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y))

train_loader = DataLoader(train_torch_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_torch_data, batch_size=1)
all_loader = DataLoader(all_torch_data, batch_size=data.shape[0])




In [29]:
# 将数据分为输入和输出
input_size = 2  # 2个输入
output_size = 1  # 1个输出
learning_rate = 1e-5 # 学习率

bpnn = Net(input_size, output_size)

optimizer = optim.Adam(bpnn.parameters(), learning_rate)  # 使用Adam算法更新参数
criteon = torch.nn.MSELoss()  # 损失函数，回归问题采用均方误差

In [30]:
# 训练模型
n_epochs = 10000
for epoch in range(n_epochs):
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = bpnn(X_batch)
        loss = criteon(y_pred, Y_batch)
        loss.backward()
        optimizer.step()

    if (epoch+1)%100 == 0:
        print(f"Epoch {epoch+1} , Loss: {loss.item()}")

Epoch 100 , Loss: 0.18516463041305542
Epoch 200 , Loss: 0.2989436686038971
Epoch 300 , Loss: 0.08429031819105148
Epoch 400 , Loss: 0.08598598092794418
Epoch 500 , Loss: 0.08052727580070496
Epoch 600 , Loss: 0.258340984582901
Epoch 700 , Loss: 0.05537844076752663
Epoch 800 , Loss: 0.13518428802490234
Epoch 900 , Loss: 0.05064769834280014
Epoch 1000 , Loss: 0.2881772220134735
Epoch 1100 , Loss: 0.002057106466963887
Epoch 1200 , Loss: 0.04716426879167557
Epoch 1300 , Loss: 0.043597035109996796
Epoch 1400 , Loss: 0.007623428478837013
Epoch 1500 , Loss: 0.17598359286785126
Epoch 1600 , Loss: 0.05194465443491936
Epoch 1700 , Loss: 0.02100137062370777
Epoch 1800 , Loss: 0.0353197380900383
Epoch 1900 , Loss: 0.031203273683786392
Epoch 2000 , Loss: 0.013626371510326862
Epoch 2100 , Loss: 0.014673823490738869
Epoch 2200 , Loss: 0.020776286721229553
Epoch 2300 , Loss: 0.10446538776159286
Epoch 2400 , Loss: 0.017527727410197258
Epoch 2500 , Loss: 0.01863020285964012
Epoch 2600 , Loss: 0.0282442346

In [31]:
# 输出验证集结果
for X_val, y_val in val_loader:
    val_y_pred = bpnn(X_val)
    loss = criteon(val_y_pred, y_val)
    print(f"预测值： {scaler.inverse_transform(val_y_pred.detach().numpy())[0][0]} ， "
          f"MSE: {loss.item()}")

预测值： 8.079035758972168 ， MSE: 0.02121681347489357
预测值： 8.338910102844238 ， MSE: 0.020164597779512405


In [32]:
# 计算整体MAPE
for X, y in all_loader:
    y_pred = bpnn(X)
    loss = criteon(y_pred, y)
    print(f"MSE: {loss.item()}")
    result = scaler.inverse_transform(y_pred.detach().numpy())

MSE: 0.00502434279769659


In [33]:
# 整体MAPE
APE = np.abs(result.reshape(-1) - data.iloc[:, 1].to_numpy()) / data.iloc[:, 1].to_numpy()
MAPE = np.mean(APE)
print(APE)
print("MAPE: ", MAPE)

[0.12015736 0.07131007 0.01908762 0.03981805 0.07182514 0.02847808
 0.00988996 0.03716177 0.05250733 0.03816531 0.097943   0.09301337]
MAPE:  0.05661308889231296


In [34]:
# 后两项MAPE
print(np.mean(APE[-2:]))
pd.DataFrame(result).to_excel('BPNN_hatx0.xlsx')

pd.DataFrame(APE).to_excel('BPNN_APE.xlsx')

0.09547818482944048


In [53]:
#print(bpnn.fc1.weight)  # 打印权重
#print(bpnn.fc1.bias)    # 打印偏置
#print(bpnn.fc2.weight)    # 打印偏置
#print(bpnn.fc2.bias)    # 打印偏置
#print(bpnn.fc3.weight)    # 打印偏置
#print(bpnn.fc3.bias)    # 打印偏置

Parameter containing:
tensor([[-0.4502, -0.2134],
        [-0.0416, -0.3045],
        [ 0.0042,  0.1918],
        [-0.5546,  0.2929],
        [-0.2783,  0.4936],
        [-0.2900,  0.1277],
        [-0.5207, -0.2343],
        [ 0.2976, -0.3975],
        [ 0.6222,  0.4245],
        [ 0.8049,  0.4589],
        [ 0.5750, -0.6856],
        [ 0.4247, -0.2020],
        [-0.4253, -0.1561],
        [ 0.5243,  0.3246],
        [ 0.2753, -0.0468],
        [ 0.7839, -0.1912]], requires_grad=True)
Parameter containing:
tensor([-3.2099e-01,  6.1485e-02, -7.0374e-01,  5.8773e-01, -4.8507e-01,
         8.5243e-02,  6.2356e-01, -1.6874e-01, -1.3178e-01, -4.3512e-01,
        -1.6186e-01, -4.7573e-01,  5.3006e-01,  2.5153e-02, -4.0095e-01,
        -7.1931e-06], requires_grad=True)
Parameter containing:
tensor([[ 0.1879, -0.0068, -0.2290, -0.0688,  0.0350, -0.1702, -0.0997, -0.1330,
          0.1562,  0.0943, -0.1645,  0.1968, -0.1094,  0.0578,  0.2455,  0.1420],
        [-0.2436, -0.1448, -0.2190,  0.00